In [11]:
import pandas as pd
import mysql.connector as mysql
import re
import numpy as np

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
    port="3306",
    database="serlo"
)

def is_flagged_word(word):
    flagged_words = [
                    'online','official','call','fantasy','contact','installation', 
                    'pay', 'manufacturer', 'delivery', 'established', 'registration',
                    'seo', 'service', 'customers', 'trading', 'company', 'buy', 'sell', 
                    'escort', 'provider', 'product', 'patients', 'website', 'sites', 
                    'free', 'affordable', 'website', 'buisness', 'installed', 'install',
                    'management', 'manufacture', 'install', 'download', 'provides',
                    'downloaded', 'services', 'development', 'drug', 'top', 'reviews',
                    'review', 'treatment', 'deliver', 'hire', 'hiring', 'supply', 'supplier', 
                    'commercial', 'discount', 'account', 'websites', 'app', 'products', 
                    'guests', 'deals'
                    ]
     
    return word.lower() in flagged_words


def is_unhelpful_word(word):
    unhelpful_words = [
                      'col','plugin','rows','state','children','content','text',
                      'type','level','children','p','h','attachment','file','layout',
                      'child','image','src','width','alt','href'
                     ]
    
    return word.lower() in unhelpful_words
        
        
def readable(words):
    result_data = ''
    for word in words:
        if word.isalpha():
            if is_unhelpful_word(word):
                pass
            else:
                result_data = result_data + ' ' + word
    return(result_data)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    return c.fetchall()

MAX_DESCRIPTION_SIZE = 1500

potential_bots = np.empty((0,2))

database = query('''SELECT id, description FROM user WHERE description != "NULL"''' )

for user in database:
    flagged_word_count = 0
    user_description = user[1]
    words = re.split('\W+', user_description)
    
    if len(words) > MAX_DESCRIPTION_SIZE:
        potential_bots = np.vstack([potential_bots, user])
        continue
    
    for word in words:
        if(is_flagged_word(word)):
            potential_bots = np.vstack([potential_bots, [user[0], readable(words)]])
            break
    
df = pd.DataFrame(potential_bots, columns = ['id', 'description'])
df.set_index('id', inplace = True)
print(df)


                                              description
id                                                       
1        As MSc Computer Science student I enjoy forwa...
34540    Arbeit bei Serlo n nIch mich im weitesten Sin...
35408    multimedia explanation Hallo allerseits mein ...
68640    Shop online for prom dresses bridesmaid dress...
70444    Wer bin ich n nHi ich bin Benni Jahre und kom...
...                                                   ...
239027   Over the years SMPL has established itself as...
239039   a https www digitaldynamicswsi com Digital Dy...
239073   An excellent option for deciding which and ho...
239090   ialah situs slot online terpopuler dengan kon...
239104   blockquote strong true a https newkhane ir we...

[99 rows x 1 columns]
